<div class="alert alert-block alert-info">
<b>Achtung:</b> Der hier angezeigt Code kann nicht direkt im Exercise Server ausgeführt werden. Der Code fuktioniert nur im <a href="http://localhost:8080/lab">Jupyter Lab der Development Environment</a>.</div>

# Übersicht - Experiment Tracking und Model Registry

In dieser Übung erweitern wir den Code, welcher unser einfaches Modell trainiert, um Experiment Tracking. Wir führen einige Experimente durch und vergleichen deren Resultate in MLFlow. Wir fügen unseren Modellen Metadaten hinzu, registrieren ein Modell in der Registry und probieren kurz den MLFlow Client (das low-level API von MLFlow aus).

# Vorbereitung

## Notebook für den Code erstellen

Kopiere als erstes dein vorheriges Notebook (`01-Simple_Model.ipynb`) mit dem Code, welcher das einfache ML Modell trainiert und gib der Kopie den Namen `02-Simple_Model_MLFlow.ipynb` und schliesse das nicht mehr verwendete Notebook `01-Simple_Model.ipynb`.

# Übungen

## MLFlow Setup

Importiere mlflow an einer geeigneten Stelle in deinem `02-Simple_Model_MLFlow.ipynb`:

In [ ]:
import mlflow

 Die Tracking URL brauchst Du nicht zu setzen. Erinnerst Du Dich, weshalb dies nicht notwendig ist?

Die URL wird im docker-compose der Development Environment als Umgebungsvariable gesetzt:

    environment:
       - MLFLOW_TRACKING_URI=http://model-registry:5001

----

Führe nun die Zellen vom Laden der Daten bis und mit Vorbereitung des finalen Testsets (`train_test_split()`) aus.

Du musst dein MLFlow Experiment benennen. Das Experiment bündelt alle Versuche mit dem Mushroom Dataset.

In [ ]:
mlflow.set_experiment("Mushroom Categorization")

----

## Hyper-Parameter definieren

Nun baust Du den bestehenden Code so um, dass die Hyper-Parameter des Classifiers vor dessen Instanzierung in ein Dictionary gespeichert werden, und dann bei der Instanzierung des Classifiers dieser Dictionary übergeben wird.

In [ ]:
# wenn wir mit der Musterlösung aus der vorherigen Übung weitermachen, könnte das zum Beispiel so aussehen:

params = {
    'kneighborsclassifier__n_neighbors': 3
}
pipeline = make_pipeline(preprocessor, clf)
pipeline.set_params(**params)

----

## Training und Evaluation

Jetzt geht es ans Evaluieren des Modells. Dazu müssen wir festlegen, welche Metriken wir verwenden (eigentlich ist dies ein sehr zentraler Schritt, welcher schon ganz am Anfang eines ML Projektes durchgeführt werden muss, denn die Metriken, mit denen wir das Modell evaluieren, müssen auf den Use Case und die für das Business relevanten Kennzahlen möglichst gut abgestimmt sein).

Wir können hier die klassische Accuracy, Precision, Recall sowie F1-Score loggen. Importiere diese Metriken von `sklearn.metrics`

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

----

Wir validieren mit einem einfachen, fixen Validation Set. Verwende dazu `train_test_split()` noch einmal auf dem Trainingsset, um von diesem noch ein fixes Validierungsset abzuspalten.

In [ ]:
X_train_small, X_val, y_train_small, y_val = train_test_split(X_train, y_train, random_state=42)

----

Damit können wir nun mit den `small` Varianten trainieren und mit `val`validieren. Bis und mit diesem Schritt gibt es keinen MLFLow-spezifischen Code.

Verwende `fit()` und `y_pred = predict()`mit X_val, um zu trainieren und predictions für die nachfolgende Validierung zu machen. 

In [ ]:
pipeline.fit(X_train_small, y_train_small)
y_pred = pipeline.predict(X_val)

----

Nun füllen wir einen Dictionary mit unseren vier Metriken.

In [ ]:
metrics_static_val_set = {
    "accuracy": accuracy_score(y_val, y_pred),
    "precision": precision_score(y_val, y_pred),
    "recall": recall_score(y_val, y_pred),
    "f1": f1_score(y_val, y_pred), 
}

Und erstelle eine MLFlow Signatur für die Trainingsdaten.

In [ ]:
from mlflow.models import infer_signature
signature = infer_signature(model_input=X_train, model_output=y_train)

----

Und jetzt, wo wir *Signatur*, *Parameter*, *Modell* und *Scores* haben, können wir den MLFlow run starten und alle vier Dinge loggen.

Als Erinnerung: Um zu vermeiden, dass bei einer Exception während des Trainings ein halb-fertiger run geloggt wird, starten wir den MLFlow run zum Loggen erst, wenn wir alle Komponenten zum Loggen beisammen haben, also erst nach Training und Evaluation.

Gib dem run den Namen. Welchen Namen Du dem run gibst, ist abhängig davon, was Du gerade ausprobierst, welche Hyperparameter Du testen möchstest usw.

In [ ]:
with mlflow.start_run(run_name="baseline model") as run:
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(
            sk_model=pipeline, signature=signature, artifact_path="mushroom"
    )

----

Wenn alles geklappt hat, sehen wir im [MLFLow UI](http://localhost:5001/), dass Metadaten, die vier Metriken, der oder die Parameter und das Modell geloggt wurden. Wenn wir auf eine Metrik klicken, wird diese als Plot angezeigt. Die Y-Achse entspricht hier dem Wert der Metrik, die X-Achse sogenanntgen *Steps*. Steps können verwendet werden, um ein Training über die Zeit abzubilden, also eine Learning Curve über Epochen, über Trainingssetgrössen oder andere Hyperparameter. Wenn wir in `log_metrics()` das `step=` Argument nicht angeben, wird standardmässig nur ein Step geloggt, was dann zum konstanten Plot führt, wie wir ihn hier sehen.

## Runs vergleichen

Ändere nun einen Parameter deines Modells und logge einen zweiten run.

In [ ]:
# code ausser die zweite Zeile und der run_name gleich wie bisher

params = {
    'kneighborsclassifier__n_neighbors': 8
}
pipeline.set_params(**params)

pipeline.fit(X_train_small, y_train_small)
y_pred = pipeline.predict(X_val)

metrics_static_val_set = {
    "accuracy": accuracy_score(y_val, y_pred),
    "precision": precision_score(y_val, y_pred),
    "recall": recall_score(y_val, y_pred),
    "f1": f1_score(y_val, y_pred), 
}

with mlflow.start_run(run_name="another_run") as run:
    mlflow.log_params(params)
    mlflow.log_metrics(metrics_static_val_set)
    mlflow.sklearn.log_model(
            sk_model=pipeline, signature=signature, artifact_path="mushroom"
    )

----

Du kannst nun im [MLFLow UI](http://localhost:5001/) beide runs selektieren und dann vergleichen. Du kannst verschiedene Plots anzeigen und die Unterschiede von Parametern und Metriken anzeigen.

## K-Fold Cross Validation

Evaluation auf einem fixen Validation Set ist nur zu empfehlen, wenn wir viel Daten haben, und unser Validation Set gross genug und damit repräsentativ für das gesamte Trainingsset ist. Kleineren Datensets und schnell trainierende Modelle erlauben eine K-fold Cross Validation und dadurch präzisere Aussagen über unsere Modell-Performance. Wie würden wir Resultate einer K-fold Cross Validation in MLFlow loggen?

Leider unterstützt MLFlow dies nicht direkt. Als Workaround könnten wir den Durchschnitt über unserer Folds loggen. 

## Autolog

Nun vereinfachen wir das Logging etwas, indem wir die autolog Funktion von MLFlow verwenden. Hierbei ist wichtig, dass die Autologging-Funktionalität aktiviert wird, *bevor* die verwendete Metrik von sklearn importiert wird.

**Starte deshalb den Jupyter Kernel neu (im Menu Kernel -> Restart).**

Führe alle notwendigen Zellen bis zum Import von MLFlow aus.

Aktiviere die Autolog Funktion gleich unterhalb der Zelle, wo du das Experiment setzt, und somit *bevor* du die Metriken von sklearn importierst.

In [ ]:
mlflow.set_experiment("Mushroom Categorization")
mlflow.autolog()

----

Führe die restlichen Zellen aus bis und mit dem `fit()` deines Modells.

Es wird autromatisch ein neuer Run geloggt, welchen du im [MLFLow UI](http://localhost:5001/) anschauen kannst. Diese Funktionalität ist sehr praktisch, da viele weitere Metadaten mitgeloggt werden.

Aber... bis jetzt wurden erst die Trainings-Metriken geloggt, was natürlich nur bedingt nützlich ist. Um auch die Validierungsmetriken zu loggen, müssen die scorer ausgeführt werden, dies wird dann von autlog gecaptured:

In [ ]:
y_pred = pipeline.predict(X_val)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

metrics_static_val_set = {
    "accuracy": accuracy_score(y_val, y_pred),
    "precision": precision_score(y_val, y_pred),
    "recall": recall_score(y_val, y_pred),
    "f1": f1_score(y_val, y_pred), 
}

## Metadaten

Runs können mit einer *Description* und mit *Tags* versehen werden. Dies kann entweder gleich während des Loggens des Runs passieren, oder nachträglich im UI oder mittels dem API (weiter unten beschrieben).

Füge mittels UI einem Run eine Beschreibung und ein neues Tag hinzu.

Dies kann in der Overview Tab eines Runs gemacht werden.

----

Auch dem Modell selber kann man Metadaten hinzufügen. Diese erscheinen dann im `MLmodel` File.

Logge einen neuen Run und füge dabei Model Metadaten hinzu. Suche diese Metadaten im UI.

In [ ]:
# dem log_model muss ein metadata Parameter mit einem dictionary hinzugefügt werden
# sichtgbar sind die Metadaten dann im Artifacts Tab im MLmodel File
mlflow.sklearn.log_model(
        sk_model=pipeline, signature=signature, artifact_path="mushroom", metadata={'foobar':'baz'}
)

----

## Models registrieren

Bis jetzt haben wir uns noch rein in der Entwicklungsphase bewegt. Wenn wir einmal zum Punkt gekommen sind, an dem wir ein Modell haben, welches wir deployen möchten, müssen wir dieses *registrieren*:

Dies können wir entweder im MLFlow UI machen, via dem Button `Register model` in der Run Ansicht. Oder wir können es direkt aus dem `log_model()` Aufruf machen, mittels dem Parameter `registered_model_name="some model name"`. Oder wir tun es mit der Funktion [mlflow.register_model()](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.register_model) oder via API (siehe unten).

Wählen wir einen Modell-Namen, welcher bisher noch nicht existiert, wird ein neues Modell in Version 1.0 registriert. Wenn wir einen Namen wählen, welcher bereits existiert, wird das Modell unter diesem Namen um eine Version höher registriert. Probiere dies einmal aus.

Registrierte Modelle erscheinen unter dem `Models` Tab. Ein Modell hat einen Namen, eine Version und Tags. Versionen können einen Alias haben, diese erklären wir gleich weiter unten.

## MLFlow Client API

Bisher haben wir mit dem higher level mlflow Modul gearbeitet. Es existiert aber noch ein zweites API, welches low level Zugriff auf MLFlow erlaubt, der [MLFlow Client](https://mlflow.org/docs/latest/python_api/mlflow.client.html), welcher ein simples CRUD Interface für Experimente, Runs und Models zur Verfügung stellt.

Verwende den MLFlow Client, um alle registrierten Modelle aufzulisten.

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()
client.search_registered_models()

----

Hole ein Modellmittels `get_model_version()` und gib dessen Description, Alias und Version aus.

In [ ]:
model = client.get_model_version(name='name of one of your models', version="1")
model.description, model.aliases, model.version

----

Füge deinem Modell mittels `update_model_version()` eine description hinzu. Die description hängt an der Version. Es gibt auch eine description, welche am Modell hängt. Prüfe danach im UI, ob dies geklappt hat.

In [ ]:
client.update_model_version(name='name of one of your models', version="1", description="foobar")

----

## Modell Versions-Aliase

Modelle können entweder via Pfad, via Name+Version oder via Alias geladen werden, vergleiche hierzu die [Dokumentation](https://mlflow.org/docs/latest/getting-started/registering-first-model/step3-load-model.html?).

Modell Versions-Aliase sind Identifier, welche an einer Version hängen. Sie werden dazu verwendet, den aufrufenden Code von der Modell-Version zu entkoppeln. Wir können unserem aktiven, deployten Modell den Alias *champion* geben und es via diesen referenzieren. Entwickeln wir ein neues, verbessertes Modell, können wir diesem den Alias *champion* zuweisen und müssen den aufrufenden Code nicht anpassen.


Registeriere dein bestes Modell, wenn noch nicht geschenhen, dann gib ihm den Alias 'champion' mittels der Funktion `set_registered_model_alias()`. Du wirst dieses Modell später noch verwenden.

In [ ]:
client.set_registered_model_alias(name='name of one of your models', version="1", alias='champion')

----

**Bitte quittiere wiederum auf [Mentimeter](https://www.menti.com/alaxbnek73eu), dass du mit der Übung durch bist**.